In [1]:
import os
import getpass

from datasets import load_dataset, Dataset, DatasetDict
from huggingface_hub import login
from transformers import AutoTokenizer
from dotenv import load_dotenv
import matplotlib.pyplot as plt

from tqdm import tqdm

In [2]:

load_dotenv('.envrc')
if 'HF_TOKEN' in os.environ:
    login(token=os.environ['HF_TOKEN'])
else:
    login(token=getpass.getpass('Huggingface token: '))

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
CHUNK_SIZE = 128
TARGET_SIZES = [16, 32, 64]

INPUT_DATASETS = [
    'MikiV/SimpleStories-SimpleStories-chunked-128-train',
    'MikiV/SimpleStories-SimpleStories-chunked-128-val',
    ]


In [4]:
def split_into_subchunks(example, chunk_size, subchunk_size):
    text = example['input_ids']
    subchunks = [
        {'input_ids': text[i*subchunk_size:(i+1)*subchunk_size]} for i in range(chunk_size // subchunk_size)
    ]
    return {'subchunks': subchunks}

In [6]:

for input_dataset_name in INPUT_DATASETS:
    for target_size in TARGET_SIZES:
        print(f"Processing dataset {input_dataset_name} into target size {target_size}...")
        input_dataset = load_dataset(input_dataset_name)
        split = lambda x: split_into_subchunks(x, CHUNK_SIZE, target_size)

        output_dataset = input_dataset['train'].map(split, batched=False, remove_columns=input_dataset.column_names, num_proc=4) # type: ignore

        assert CHUNK_SIZE % target_size == 0
        output_dataset_name = input_dataset_name.replace('chunked', 'subchunked').replace(str(CHUNK_SIZE), f'{target_size}x{CHUNK_SIZE // target_size}')

        output_dataset.push_to_hub(output_dataset_name, private=True)

Processing dataset MikiV/SimpleStories-SimpleStories-chunked-128-train into target size 16...


EmptyDatasetError: The directory at hf://datasets/MikiV/SimpleStories-SimpleStories-chunked-128-train@eac2614db5134e6e5067806db0c16ce1efc1d2d7 doesn't contain any data files